This notebook is called backup due to three reasons
1. The nature of LSTM (input, output and forget gates) is to learn from independent sequential data. Plain LSTM learns that everything is important to survive until last timestep, while using _attention_ we teach LSTM what timesteps matter most and to average them, this helps avoid the need of mean rolling windows, and results in a faster training.

2. Rolling windows helps to reduce entropy per timestep and to denoise high-frequency fluctuations, but also are _cheap_ to compute and aim to a better generalization. Meanwhile using attention requires maintanance itself, as this new "function" may put attention when the data changes a little

3. Rolling windows in attention + LSTM naturally makes the model to memorize the clipped RUL, which is a expected result.

However, for production a stable, observative and easy to debug version is better. Before was seen with some experiments, that plain LSTM has lower variance, and also provides important information to the user (moving average).

So in conclusion, using attention seems a good and viable option, which helps to diagnoise our data (attention describes it). But is lack of the simplicity of plain LSTM, which is more valuable in a environment where we need stability and observability. 

# Libraries

In [ ]:
# -- Standard Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# -- Model --
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, Dataset
# -- Error Metrics --
from sklearn.metrics import mean_squared_error, r2_score
# -- Model Selection --
from sklearn.model_selection import train_test_split

Fix description of data

## Data Description

In [ ]:
# Give names to the features
index_names = ['engine', 'cycle']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_info = [ 
"(Fan inlet temperature) (◦R)",
"(LPC outlet temperature) (◦R)",
"(HPC outlet temperature) (◦R)",
"(LPT outlet temperature) (◦R)",
"(Fan inlet Pressure) (psia)",
"(bypass-duct pressure) (psia)",
"(HPC outlet pressure) (psia)",
"(Physical fan speed) (rpm)",
"(Physical core speed) (rpm)",
"(Engine pressure ratio(P50/P2)",
"(HPC outlet Static pressure) (psia)",
"(Ratio of fuel flow to Ps30) (pps/psia)",
"(Corrected fan speed) (rpm)",
"(Corrected core speed) (rpm)",
"(Bypass Ratio) ",
"(Burner fuel-air ratio)",
"(Bleed Enthalpy)",
"(Required fan speed)",
"(Required fan conversion speed)",
"(High-pressure turbines Cool air flow)",
"(Low-pressure turbines Cool air flow)" 
]

def name_sensor_data(sensor_info: list):
    '''Helper to describe sensor data and get column names'''
    sensor_dict = {}
    for i, sensor in enumerate(sensor_info):
        sensor_dict['s_'+ str(i + 1)] = sensor

    sensor_names = list(sensor_dict.keys())
    col_names = index_names + setting_names + sensor_names
    return sensor_dict, col_names

sensor_description, col_names = name_sensor_data(sensor_info)

In [ ]:
from pathlib import Path

data_path = Path("..") / "data" / "CMaps"


df_train = pd.read_csv(data_path / "train_FD001.txt",
                       sep=r'\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv( data_path / "test_FD001.txt",
                      sep=r'\s+', header=None, index_col=False, names=col_names)
y_valid = pd.read_csv(data_path /"RUL_FD001.txt",
                     sep=r'\s+', header=None, names=['RUL'])
print(f"Test Shape: {df_test.shape}\nTrain Shape:{df_train.shape}\nRUL Labels: {y_valid.shape}")
print(f"Percentage of the test-set: {len(df_test)/(len(df_train)+len(df_test))*100:.3f}%")

### Detect nulls and add RUL

In [ ]:
def detect_null(dfs: list):
    results = []
    for i, df in enumerate(dfs):
        null_mask = df.isnull().any(axis=1)
        null_row_idx = np.where(null_mask)[0]
        null_counts = df.isnull().sum(axis=1)
        results.append((i, null_row_idx, null_mask, null_counts))
        print(f"Dataframe {i}:")
        print(f"|   Total tows with nulls: {len(null_row_idx)}")
        print(f"|   Total null values {df.isnull().sum().sum()}")
        if len(null_row_idx) > 0:
            print(f"  Rows with nulls: {null_row_idx.tolist()[:10]}")  
            if len(null_row_idx) > 10:
                print(f"  ... and {len(null_row_idx) - 10} more")
        else:
            print(f"|-->No null values detected!\n")

    return results

null_report = detect_null([df_train, df_test, y_valid])

In [ ]:
def add_RUL(
        df, 
        unit_col='engine', 
        cycle_col='cycle', 
        rul_col='RUL', 
        clip_max=None
    ):
    max_cycles = df.groupby(unit_col)[cycle_col].transform('max')
    df[rul_col] = max_cycles - df[cycle_col]
    if clip_max is not None:
        # 1. clipping helps the model to have a narrower prediction space, 
        # using clip(upper) we hope the model gets a high value but at the same time does not overstimate the RUL
        # i.e., a machine could fail if we expect it to fail at 200 but fails at 190 for some reason
        # so we are telling the model to expect less, like a quantile regression
        # https://www.kaggle.com/code/wassimderbel/nasa-predictive-maintenance-rul
        df[rul_col] = df[rul_col].clip(upper=clip_max) # 
    return df

df_train = add_RUL(df_train, clip_max=130)
df_train[['cycle','RUL']].head(5)

In [ ]:
df_train.describe().transpose()

Point 1 is to be observed, the mean RUL is dependent of the clipping, but usually is not expected to have a high RUL (75\% of the data is at most 155), even if clipping is removed, the mean is not high, so by configurating this parameter we can enhance the learning capability of the LSTM.

In [ ]:
RUL_column = df_train.RUL
mean_rul = RUL_column.mean()
std_rul = RUL_column.std()

plt.figure(figsize=(6, 6))
counts, bins, _ = plt.hist(df_train['RUL'], bins='rice', color="#159099")
plt.vlines(mean_rul, ymin=0, ymax=max(counts), color='r')
plt.vlines([mean_rul - std_rul, mean_rul + std_rul], ymin=0, ymax=max(counts), 
           color='black', linestyles='dashed')
plt.show()

print(RUL_column.describe())

# Model Performance

In [ ]:
from pprint import pp
def evaluate(
    y_true,
    y_pred,
    label: str = ''
):
    """Evaluation function"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    pp({
        "label": label,
        "RMSE": rmse,
        "R2": r2
    })

def plot_performance(
        y_true: torch.Tensor, 
        y_pred: torch.Tensor,
        max_engines: int = None,
        figsize: tuple = ((25,10))
    ):
    '''Visualization of predicted vs actual RUL'''
    
    fig, ax = plt.subplots(figsize=figsize)

    y_true_full = np.array(y_true).flatten()
    y_pred_full = np.array(y_pred).flatten()

    if max_engines is not None:
        y_true = y_true_full[:max_engines].astype(float)
        y_pred = y_pred_full[:max_engines].astype(float)


    indices = np.arange(len(y_true))
    bar_width = 0.35

    ax.bar(
        x=indices - bar_width / 2, 
        height=y_true,
        width=bar_width,
        color='#2ecc71',
        alpha=0.8,
        label='[True] RUL'
    )
    ax.bar(
        x=indices + bar_width / 2,
        height=y_pred,
        width=bar_width,
        color='#e74c3c',
        alpha = 0.8,
        label='[Pred] RUL',
    )
    
    # Showtime
    for i, (true, pred) in enumerate(zip(y_true, y_pred)):
        ax.plot(
            [i - bar_width / 2, i + bar_width / 2],
            [true, pred],
            color='#34495e',
            ls=':',
            alpha=0.6
        )
        
    ax.set_title('RUL Prediction Performance', fontsize=20, pad=20)
    ax.set_xlabel('Engine Index', fontsize=16)
    ax.set_ylabel('RUL (cycles)', fontsize=16)
    ax.set_xticks(indices)
    ax.legend(framealpha=1,)
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

## Clean data, no useless sensors 
At first, I tried to use a LSTM with (256, 256) hidden units, the problem was that it needed a rolling mean to get good results, that is because the previous architecture used a last-head strategy, in which only last information was considered for training. Using attention this need is avoided because attention provides importance to pikes or low signals that provide more information during training  

In [ ]:
drop_cols = [
    "setting_1",
    "setting_2",
    "setting_3",
    "cycle",
    "s_1", "s_5", "s_6","s_10", "s_16", "s_18", "s_19"
]

df_clean = df_train.drop(columns=drop_cols)
feature_columns = [col for col in df_clean.columns]

def calculate_rolling_mean(df, window=1):
    '''Calculates rolling features'''
    df = df.copy()

    sensor_cols = [c for c in df.columns if c.startswith("s_")]

    if window > 1:
        for sensor in sensor_cols:
            df[f"{sensor}_rm"] = (
                df.groupby("engine")[sensor]
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
        return df
    else:
        return df

df_rm = calculate_rolling_mean(df_clean)
if any(col.endswith('_rm') for col in feature_columns):
    sample_engine = df_rm[df_rm['engine'] == 1]
    sample_engine[['s_17', 's_17_rm']].plot(figsize=(12,6));

In [ ]:
df_rm.drop(['engine'], axis=1, inplace=True)
X_clean = df_rm.drop(["RUL"], axis=1)
y_clean = df_rm["RUL"]
feature_columns = list(X_clean.columns)
input_dim = len(feature_columns)
print(f"Input dim: {input_dim} \n Feature Columns: {feature_columns}")

In [ ]:
def _get_sequences(
        X: pd.DataFrame,
        y: pd.Series,
        SEQ_LEN: int
    ):
    X_np = np.array(X)
    y_np = np.array(y)
    X_seq, y_seq = [], []
    for i in range(len(X_np) - SEQ_LEN):
        X_seq.append(X_np[i:i+SEQ_LEN])
        y_seq.append(y_np[i+SEQ_LEN])
    X_seq = torch.tensor(X_seq, dtype=torch.float32)
    y_seq = torch.tensor(y_seq, dtype=torch.float32).unsqueeze(1)
    return X_seq, y_seq

SEQ_LEN = 30
X_seq, y_seq = _get_sequences(
    X_clean,
    y_clean,
    SEQ_LEN=SEQ_LEN
)
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, y_seq, shuffle=False
)

mean = X_train_seq.mean(dim=(0, 1), keepdim=True)  # (1, 1, F)
std  = X_train_seq.std(dim=(0, 1), keepdim=True) + 1e-8
X_train_seq = (X_train_seq - mean) / std
X_test_seq  = (X_test_seq  - mean) / std

def arr_to_tensor(data: np.ndarray | pd.DataFrame):
    '''Converts from pd.DataFrame/np.ndarray to torch.tensor'''
    if not isinstance(data, (pd.DataFrame, np.ndarray)):
        raise TypeError("data can't be converted to tensor")
    if isinstance(data, np.ndarray):
        return torch.tensor(data, dtype=torch.float32)
    if isinstance(data, pd.DataFrame):
        return torch.tensor(data.to_numpy(), dtype=torch.float32)
    
def get_device(device: torch.device | str = "auto") -> torch.device:
    """
    :param device: One for 'auto', 'cuda', 'cpu'
    :return: supported PyTorch device
    """
    if device == "auto":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.device(device)

class LSTMRegressor(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 256,
        num_layers: int = 2, # fast and secure maintanance n_layers > 3 consumes more time
        output_dim: int = 1,
        dropout_prob: float = 0.3,
        device: torch.device | str = "auto"
    ):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        self.device = get_device(device)
        
        self.lstm = nn.LSTM(
            input_size=self.input_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            dropout=self.dropout_prob if self.num_layers > 1 else 0.0,
            batch_first=True,
            device = self.device
        )

        self.attn_score = nn.Linear(self.hidden_dim, 1, device=self.device)

        # Regression head
        self.head = nn.Linear(self.hidden_dim, self.output_dim, device=self.device)

    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch, seq_len, hidden_dim)

        # Compute attention scores and weights
        scores = self.attn_score(lstm_out)  # (batch, seq_len, 1)
        attn_weights = torch.softmax(scores, dim=1)  # (batch, seq_len, 1)

        # Weighted sum over time
        context = torch.sum(attn_weights * lstm_out, dim=1)  # (batch, hidden_dim)

        # Final prediction
        return self.head(context)

print(f"Training using a sequence of length {SEQ_LEN}, running in {get_device()}")
print(LSTMRegressor(input_dim))

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, X_seq, y_seq):
        self.X = X_seq
        self.y = y_seq

        assert self.X.dim() == 3
        assert self.y.dim() == 2
        assert len(self.X) == len(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = SequenceDataset(X_train_seq, y_train_seq)

train_loader = DataLoader(
    train_ds,
    batch_size=256,     
    shuffle=True,
    drop_last=True
)

device = get_device()
model = LSTMRegressor(input_dim=input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 500

history = []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    mean_loss = total_loss / len(train_loader)
    history.append(mean_loss)

    if (epoch + 1) % 10 == 0:
        print(f"> Epoch {epoch+1}/{epochs}, loss = {mean_loss:.4f}")


In [ ]:
def plot_learning_curve(history: list):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(
        range(len(history)), 
        history,
        color='red'
    )
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss (MSE)')
    ax.grid(axis='both', alpha=0.5, ls=':')
    plt.show()
plot_learning_curve(history)

In [ ]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'input_dim': model.input_dim,
    'hidden_dim': model.hidden_dim,
    'num_layers': model.num_layers,
    'dropout_prob': model.dropout_prob, 
    'device': device,
    'normalization_stats': {
        'mean': mean,  
        'std': std     
    }
}

save_dir = Path("../checkpoints")   # <- whatever folder you want
save_dir.mkdir(parents=True, exist_ok=True)

checkpoint_path = save_dir / "lstm_model_inference_backup.pth"

torch.save(checkpoint, checkpoint_path)
print("Checkpoint saved correctly!")

# Test Prediction

In [ ]:
checkpoint = torch.load(checkpoint_path, weights_only=True)

# Instantiate the model using the saved hyperparameters
model_for_testing = LSTMRegressor(
    input_dim=checkpoint['input_dim'],
    hidden_dim=checkpoint['hidden_dim'],
    num_layers=checkpoint['num_layers'],
    dropout_prob=checkpoint.get('dropout_prob', 0.2) # .get for backward compatibility
).to('cpu')

# Load the trained weights
model_for_testing.load_state_dict(checkpoint['model_state_dict'])


# This turns off dropout and other training-specific layers.
model_for_testing.eval()

print("Model loaded successfully from checkpoint for testing.")

# The test set `X_test_seq` is already normalized, so we can use it directly.
# Use torch.no_grad() to make predictions more efficient (no gradients needed)
with torch.no_grad():
    y_pred_tensor = model_for_testing(X_test_seq)

# Convert predictions and ground truth to NumPy arrays for plotting
# .cpu() is important if trained on a GPU
y_pred_np = y_pred_tensor.cpu().numpy()
y_true_np = y_test_seq.cpu().numpy()


print(evaluate(y_true_np, y_pred_np, type(model).__name__))

SAMPLES_TO_PLOT = 50

print(f"Plotting performance for the first {SAMPLES_TO_PLOT} test samples...")

plot_performance(
    y_true=y_true_np, 
    y_pred=y_pred_np,
    max_engines=SAMPLES_TO_PLOT
)

## Final valid testing

In [ ]:
SEQ_LEN = 30 # The same sequence length you used for training
df_test_rm = calculate_rolling_mean(df_test)
feature_columns_rm = feature_columns + [col for col in df_rm if col.endswith('rm')]

X_test_sequences = []
for engine_id in df_test_rm['engine'].unique():
    # Get the data for the current engine
    engine_data = df_test_rm[df_test_rm['engine'] == engine_id]
    
    # Extract the features for that engine
    engine_features = engine_data[feature_columns]
    last_sequence = engine_features.tail(SEQ_LEN).to_numpy()
    
    X_test_sequences.append(last_sequence)

X_test_final_np = np.array(X_test_sequences)

X_test_final = torch.tensor(X_test_final_np, dtype=torch.float32)

print(f"Shape of the final test sequences tensor: {X_test_final.shape}")

In [ ]:
checkpoint = torch.load(checkpoint_path, weights_only=True)

mean = checkpoint['normalization_stats']['mean']
std = checkpoint['normalization_stats']['std']

# Apply the normalization
X_test_final_normalized = (X_test_final - mean) / std

# Make predictions on the final, prepared test set
with torch.no_grad():
    y_pred_final_tensor = model_for_testing(X_test_final_normalized)

# Convert predictions and ground truth to NumPy arrays
y_pred_final_np = y_pred_final_tensor.cpu().numpy()
y_true_final_np = y_valid.to_numpy() # y_valid is the RUL_FD001.txt data

evaluate(y_true_final_np, y_pred_final_np, type(model).__name__)

# Visualize the results using your plotting function
print("Plotting performance on the final hold-out test set...")
plot_performance(
    y_true=y_true_final_np, 
    y_pred=y_pred_final_np,
    max_engines=SAMPLES_TO_PLOT
)

log: 'hidden_dim' = 128, got 0.78 of r2